In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-_8nykgvf
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-_8nykgvf
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [3]:
!pip install bnlp_toolkit

  Preparing metadata (setup.py) ... done
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached charset_normalizer-3.4.4-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 10.3 MB/s  0:00:02 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 8.7 MB/s  0:00:03m 8.9 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 8.1 MB/s  0:00:02m 8.5 MB/s eta 0:00:01
   ━━━━━━━━

In [7]:
!pip install torch -q

ERROR: Operation cancelled by user
^C
Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7b32e0fe55a0>
Traceback (most recent call last):
  File "/home/fabiha-kraitlabs/miniconda3/envs/perturbation_env/lib/python3.10/tempfile.py", line 446, in __del__
    self.close()
  File "/home/fabiha-kraitlabs/miniconda3/envs/perturbation_env/lib/python3.10/tempfile.py", line 442, in close
    unlink(self.name)
KeyboardInterrupt: 


In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
import numpy as np
from normalizer import normalize
from bnlp import BasicTokenizer
from tqdm import tqdm
import string
import pandas as pd

/home/fabiha-kraitlabs/miniconda3/envs/perturbation_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /home/fabiha-
[nltk_data]     kraitlabs/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
bangla_punctuations = ["।", "৺", ",", ";", ":", "?", '"', "…", "(", ")", "{", "}", "[", "]", "—", "!", "৻", "॥" ]

In [ ]:
class BanglaBertAttentionAnalyzer:
    def __init__(self, model_name="sagorsarker/bangla-bert-base", prob):
        self.prob = prob
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
        self.model = AutoModel.from_pretrained(model_name)
        # self.model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")
        self.model.to(self.device)
        self.model.eval()

    def get_attention_scores(self, text):
        text = normalize(text)
        
        basic_tokenizer = BasicTokenizer()
        pre_tokens = basic_tokenizer.tokenize(text)
        pre_tokenized_text = " ".join(pre_tokens)
        
        # Tokenize input text
        inputs = self.tokenizer(
            pre_tokenized_text, 
            return_tensors="pt", 
            add_special_tokens=True,
            max_length=512,  # Set maximum length
            truncation=True  # Enable truncation
        )
        
        
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Get model outputs with attention scores
        with torch.no_grad():
            outputs = self.model(**inputs, output_attentions=True)

        # Get attention scores from all layers and heads
        attention = outputs.attentions  # tuple of tensors (n_layers, batch_size, n_heads, seq_len, seq_len)
       
        # Average attention scores across all layers and heads
        attention_scores = torch.mean(torch.stack(attention), dim=(0, 2))  # (batch_size, seq_len, seq_len)
        
        attention_scores = attention_scores.cpu().numpy()[0]  # (seq_len, seq_len)
        
        # Get token-wise attention scores by averaging attention given to each token
        token_attention = attention_scores.mean(axis=0)  # (seq_len,)

        # Get original tokens
        tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

        # print("tokens ", tokens)
        tokens = [normalize(token) for token in tokens]
        return tokens, token_attention

    def get_salient_words(self, text):
        top_k = max(1, round(len(text.split()) * self.prob))
        # print(top_k)
        tokens, attention_scores = self.get_attention_scores(text)

        # Create word-level scores by combining subword tokens
        word_scores = {}
        current_word = ""
        current_score = 0
        
        for token, score in zip(tokens, attention_scores):
            # print("token ",token)
            # print("score ",score)
            if token.startswith("##"):
                current_word += token[2:]
                current_score += score
            else:
                if current_word:
                    word_scores[current_word] = current_score
                current_word = token
                current_score = score
                
        # Add the last word
        if current_word:
            word_scores[current_word] = current_score

        remove_special_token = ['[CLS]', '[SEP]', '[PAD]'] + bangla_punctuations
        # Remove special tokens
        for special_token in remove_special_token:
            word_scores.pop(special_token, None)

        # print(word_scores)
        # Get top-k salient words
        salient_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return salient_words

    def visualize_attention(self, text):
        tokens, attention_scores = self.get_attention_scores(text)
        
        # Normalize attention scores to [0, 1]
        normalized_scores = (attention_scores - attention_scores.min()) / (attention_scores.max() - attention_scores.min())
        
        # Create HTML with highlighting
        html_text = ""
        for token, score in zip(tokens, normalized_scores):
            if token in ['[CLS]', '[SEP]', '[PAD]']:
                continue
            if token.startswith("##"):
                token = token[2:]
            intensity = int(score * 255)
            html_text += f'<span style="background-color: rgba(255, 165, 0, {score:.2f})">{token}</span>'
            
        return html_text


In [54]:
# Initialize the analyzer

analyzer = BanglaBertAttentionAnalyzer()
# Example Bangla text (you can replace this with your own text)
text = "আমি বাংলায় প্রোগ্রামিং শিখছি।"

# Get the most salient words
salient_words = analyzer.get_salient_words(text)
print("Most salient words:")
for word, score in salient_words:
    print(f"{word}: {score:.4f}")

# Visualize attention in HTML (useful in Jupyter notebooks)
# html_output = analyzer.visualize_attention(text)

Most salient words:
শিখছি: 0.2401
প্রোগ্রামিং: 0.1435
বাংলায়: 0.1308
আমি: 0.1210


In [55]:
analyzer = BanglaBertAttentionAnalyzer()
text = "কথা বলা যাবে চ্যাটজিপিটির সঙ্গে"

salient_words = analyzer.get_salient_words(text)
print("Most salient words:")
for word, score in salient_words:
    print(f"{word}: {score:.4f}")


Most salient words:
চ্যাটজিপিটির: 0.4120
যাবে: 0.1071
বলা: 0.1054
সঙ্গে: 0.1016
কথা: 0.0880


In [101]:
df=pd.read_csv('data/xl_sum/xl_sum_dataset.csv')
df = df[(df['sentence_count'] > 3)]

In [102]:
df.columns

Index(['id', 'url', 'title', 'summary', 'text', 'text_length_char',
       'text_length_word', 'sentence_count', 'rand_word_idx', 'rand_sent_idx',
       'salient_words', 'salient_words_scores', 'perturbed_text_words',
       'perturbed_text_sentences', 'perturbed_text_salient'],
      dtype='object')

In [103]:
english_punctuation = set(string.punctuation)
bangla_punctuation = set("॥।၊‐‑‒–—―‘’“”•…‧‰′″‹›‼‽⁄⁊⸘⸙⸚⸛⸜⸝⸞⸟⸠⸡⸢⸣⸤⸥⸮⸰⸱⸲⸳⸴⸵⸶⸷⸸⸹⸺⸻⸼⸽⸾⸿、。〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〽゠・︐︑︒︓︔︕︖︗︘︙︰︱︲︳︴︵︶︷︸︹︺︻︼︽︾︿﹀﹁﹂﹃﹄﹅﹆﹇﹈﹉﹊﹋﹌﹍﹎﹏﹐﹑﹒﹔﹕﹖﹗﹘﹙﹚﹛﹜﹝﹞﹟﹠﹡﹢﹣﹤﹥﹦﹨﹩﹪﹫！＂＃＄％＆＇（）＊＋，－．／：；＜＝＞？＠［＼］＾＿｀｛｜｝～｟｠｡｢｣､･")

# Combine all punctuation into a single set
all_punctuation = english_punctuation.union(bangla_punctuation)

In [104]:
# df = df[:5]

In [ ]:
p = 0.2
analyzer = BanglaBertAttentionAnalyzer(prop=p)

In [107]:
words_list = []
scores_list = []

for text in df['text'].astype(str):
    salient_words = analyzer.get_salient_words(text)  # This returns list of (word, score) tuples
    # Create separate lists for words and scores for this specific text
    text_words = []
    text_scores = []
    
    for word, score in salient_words:
        if word in all_punctuation:
            continue
        text_words.append(word)
        text_scores.append(score)
        
    # Append these lists to our main lists
    words_list.append(text_words)
    scores_list.append(text_scores)

# Now you can add to dataframe
df['salient_words'] = words_list
df['salient_words_scores'] = scores_list

In [108]:
df

,id,url,title,summary,text,text_length_char,text_length_word,sentence_count,rand_word_idx,rand_sent_idx,salient_words,salient_words_scores,perturbed_text_words,perturbed_text_sentences,perturbed_text_salient
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,2893,423,28,"[355, 220, 370, 390, 246, 367, 380, 238, 42, 4...","[0, 2, 25, 6, 14, 10]","[বেশ, উইগুরদের, শিনজিয়াংয়ে, অননুমোদিত, কনস্য...","[0.010106517, 0.0093539795, 0.008652595, 0.007...",arrest kin আইডি কার্ড হাতে উইগুর নারীদের বিক্ষ...,dhokkho shonshokder id card hate Uyghur naride...,আটক স্বজনদের আইডি কার্ড হাতে Wigur নারীদের বিক...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",1987,282,16,"[113, 261, 279, 106, 160, 232, 241, 142, 56, 1...","[9, 8, 3]","[জাল্লিকাটু, এফটিএনটিএ, এসোসিয়েশনগুলো, সুপারম...","[0.010910931, 0.010729774, 0.007918667, 0.0076...","ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...","ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...","ভারতের অন্য zoneo কোক , pepsi নিষিদ্ধ করার দাব..."
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,3499,526,39,"[303, 465, 277, 484, 79, 470, 204, 413, 258, 5...","[0, 7, 25, 12, 33, 14, 26, 11]","[বেশ, ইনডোর, বাহিত, ডেঙ্গুতে, জটিলতাগুলো, হাসপ...","[0.008970732, 0.004685824, 0.004479605, 0.0043...",Suhrawardy হাসপাতালের picture সাধারণত সেপ্টেম্...,Suhrawardy hospitaler chitra sadharonto Septem...,সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,5067,749,39,"[273, 546, 218, 300, 243, 54, 739, 650, 423, 4...","[5, 37, 8, 36, 21, 20, 26, 30]","[ক্লাউসুটিসের, অঙ্গরাজ্যগুলোর, কারচুপির, শংকিত...","[0.00837544, 0.0064076437, 0.0059419577, 0.005...",ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,ডোনাল্ড ট্রাম্প অভিযোগ karsen টু্‌ইটার আমেরিকা...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,3110,463,40,"[91, 104, 252, 225, 207, 113, 231, 263, 279, 3...","[21, 10, 12, 20, 28, 30, 24, 16]","[বেশ, জনদুর্ভোগ, বানভাসী, মেঘালয়, আশ্রয়কেন্দ...","[0.010410082, 0.008789574, 0.0066059674, 0.006...",বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে।বাংলাদেশ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে । বাংলাদ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,news-51306016,https://www.bbc.com/bengali/news-51306016,"করোনাভাইরাস: মহামারির আশঙ্কা, বিশ্বজুড়ে আসতে ...",করোনাভাইরাসে আক্রান্ত হয়ে মৃত্যুর সংখ্যা ১৭০ ...,বিশ্ব স্বাস্থ্য সংস্থা বলছে মহামারী আকারে পুরো...,1672,251,13,"[11, 10, 173, 211, 93, 191, 212, 42, 100, 242,...","[8, 0, 11]","[ইনফ্লুয়েঞ্জা, গ্যাব্রিয়েসুস, ৭৭১১জন, বহির্গ...","[0.012661827, 0.011186989, 0.010765303, 0.0097...",বিশ্ব স্বাস্থ্য সংস্থা বলছে মহামারী in shape প...,Bishwo hospital somporka boleche protishodha e...,বিশ্ব স্বাস্থ্য সংস্থা বলছে মহামারী আকারে পুরো...
1008,160804_bangladesh_india_wild_elephant_rescue,https://www.bbc.com/bengali/news/2016/08/16080...,বুনো হাতিটির পেছনে ভারত-বাংলাদেশের বনকর্মীরা,বাংলাদেশের জামালপুরের একটি চরে আটকে থাকা ভারতী...,জামালপুরের প্রত্যন্ত চরে আটকে রয়েছে ভারতীয় এ...,2735,420,30,"[411, 326, 154, 325, 190, 303, 85, 301, 58, 21...","[18, 22, 15, 27, 24, 6]","[ট্রাঙ্কুলাইজিং, নদীনালা, কৃষ্টপুর, বনবিভাগের,...","[0.0076565137, 0.006286313

In [109]:
df_rnd = pd.read_csv('data/prob_20/xl_sum_20.csv')

In [110]:
df['rand_sent_idx'] = df_rnd['rand_sent_idx']
df['rand_word_idx'] = df_rnd['rand_word_idx']
df.drop(columns=['perturbed_text_words'], inplace=True)

In [111]:
df.to_csv("data/prob_20/xl_sum_20.csv", index = False)